Bru's notebook

In [37]:
import pandas as pd
import numpy as np
import re

In [38]:
import sys
sys.path.append('../../utils')
import functions

In [39]:
clean_df = pd.read_csv('../../data/raw/Clean_Dataset.csv')
business_df = pd.read_csv('../../data/raw/business.csv')
economy_df = pd.read_csv('../../data/raw/economy.csv')

In [40]:
functions.show_basic_info(clean_df)


DataFrame Shape: (300153, 12)
Number of Rows: 300153
Number of Columns: 12

Data Types of Columns:
Unnamed: 0            int64
airline              object
flight               object
source_city          object
departure_time       object
stops                object
arrival_time         object
destination_city     object
class                object
duration            float64
days_left             int64
price                 int64
dtype: object

Missing Values per Column:
Unnamed: 0          0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

First 5 Rows of Data:
   Unnamed: 0   airline   flight source_city departure_time stops  \
0           0  SpiceJet  SG-8709       Delhi        Evening  zero   
1           1  SpiceJet  SG-8157       Delhi  Early_Morning  zero   
2           2  

In [41]:
functions.show_basic_info(business_df)


DataFrame Shape: (93487, 11)
Number of Rows: 93487
Number of Columns: 11

Data Types of Columns:
date          object
airline       object
ch_code       object
num_code       int64
dep_time      object
from          object
time_taken    object
stop          object
arr_time      object
to            object
price         object
dtype: object

Missing Values per Column:
date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
dtype: int64

First 5 Rows of Data:
         date    airline ch_code  num_code dep_time   from time_taken  \
0  11-02-2022  Air India      AI       868    18:00  Delhi    02h 00m   
1  11-02-2022  Air India      AI       624    19:00  Delhi    02h 15m   
2  11-02-2022  Air India      AI       531    20:00  Delhi    24h 45m   
3  11-02-2022  Air India      AI       839    21:25  Delhi    26h 30m   
4  11-02-2022  Air India      AI       544    17:15 

In [42]:
functions.show_basic_info(economy_df)


DataFrame Shape: (206774, 11)
Number of Rows: 206774
Number of Columns: 11

Data Types of Columns:
date          object
airline       object
ch_code       object
num_code       int64
dep_time      object
from          object
time_taken    object
stop          object
arr_time      object
to            object
price         object
dtype: object

Missing Values per Column:
date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
dtype: int64

First 5 Rows of Data:
         date   airline ch_code  num_code dep_time   from time_taken  \
0  11-02-2022  SpiceJet      SG      8709    18:55  Delhi    02h 10m   
1  11-02-2022  SpiceJet      SG      8157    06:20  Delhi    02h 20m   
2  11-02-2022   AirAsia      I5       764    04:25  Delhi    02h 10m   
3  11-02-2022   Vistara      UK       995    10:20  Delhi    02h 15m   
4  11-02-2022   Vistara      UK       963    08:50  Del

# Data Cleaning

## Rename Columns

In [43]:
clean_df.drop(columns=['Unnamed: 0', 'days_left'], inplace=True) # drop duplicate index and days left columns

In [44]:
replacement_dict = {
    'airline': 'airline_name',
    'flight':'flight_code',
    'source_city': 'departure_city',
    'duration': 'flight_duration',
    'destination_city': 'arrival_city',
    'duration':'flight_duration',
    'price': 'price'
}

clean_df = functions.rename_columns(clean_df, replacement_dict)
clean_df.head()

,airline_name,flight_code,departure_city,departure_time,stops,arrival_time,arrival_city,class,flight_duration,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,5955


In [45]:
replacement_dict = {
    'date': 'flight_date',
    'airline': 'airline_name',
    'ch_code': 'airline_code',
    'num_code': 'flight_number',
    'dep_time': 'departure_time',
    'from': 'departure_city',
    'time_taken': 'flight_duration',
    'stop': 'stops',
    'arr_time': 'arrival_time',
    'to': 'arrival_city',
    'price': 'price'
}

business_df = functions.rename_columns(business_df, replacement_dict)
business_df.head()

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690"


In [46]:
replacement_dict = {
    'date': 'flight_date',
    'airline': 'airline_name',
    'ch_code': 'airline_code',
    'num_code': 'flight_number',
    'dep_time': 'departure_time',
    'from': 'departure_city',
    'time_taken': 'flight_duration',
    'stop': 'stops',
    'arr_time': 'arrival_time',
    'to': 'arrival_city',
    'price': 'price'
}

economy_df = functions.rename_columns(economy_df, replacement_dict)
economy_df.head()

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


## Remove duplicate rows

In [47]:
functions.remove_duplicates(clean_df)

Removed 230838 duplicate rows.


,airline_name,flight_code,departure_city,departure_time,stops,arrival_time,arrival_city,class,flight_duration,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,5955
...,...,...,...,...,...,...,...,...,...,...
300069,Air_India,AI-545,Chennai,Night,zero,Night,Hyderabad,Business,1.33,15000
300100,Vistara,UK-838,Chennai,Night,one,Morning,Hyderabad,Business,12.33,58032
300101,Vistara,UK-832,Chennai,Early_Morning,one,Morning,Hyderabad,Business,26.33,58032
300110,Vistara,UK-834,Chennai,Evening,one,Morning,Hyderabad,Business,16.00,87118


In [48]:
functions.remove_duplicates(business_df)

Removed 0 duplicate rows.


,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690"
...,...,...,...,...,...,...,...,...,...,...,...
93482,31-03-2022,Vistara,UK,822,09:45,Chennai,10h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:50,Hyderabad,"69,265"
93483,31-03-2022,Vistara,UK,826,12:30,Chennai,10h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:55,Hyderabad,"77,105"
93484,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"79,099"
93485,31-03-2022,Vistara,UK,828,07:00,Chennai,10h 00m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,17:00,Hyderabad,"81,585"


In [49]:
functions.remove_duplicates(economy_df)

Removed 2 duplicate rows.


,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"
...,...,...,...,...,...,...,...,...,...,...,...
206769,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,697"
206770,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,709"
206771,31-03-2022,Vistara,UK,826,12:30,Chennai,20h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,09:05,Hyderabad,"8,640"
206772,31-03-2022,Vistara,UK,822,09:45,Chennai,23h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,09:05,Hyderabad,"8,640"


## Data Standardization
- Ensuring matching data types
- Formatting values consistently
- Normalizing textual data

## Standardize 'price' column
Convert data type from string to integer in business and economy dataframes

In [50]:
def standardize_price(df, column_name='price'):
    df[column_name] = df[column_name].str.replace(',', '', regex=True).astype(int)
    return df

business_df = standardize_price(business_df, 'price')
economy_df = standardize_price(economy_df, 'price')

print('Business DF:')
print(business_df)
print('\nEconomy DF:')
print(economy_df)

Business DF:
      flight_date airline_name airline_code  flight_number departure_time  \
0      11-02-2022    Air India           AI            868          18:00   
1      11-02-2022    Air India           AI            624          19:00   
2      11-02-2022    Air India           AI            531          20:00   
3      11-02-2022    Air India           AI            839          21:25   
4      11-02-2022    Air India           AI            544          17:15   
...           ...          ...          ...            ...            ...   
93482  31-03-2022      Vistara           UK            822          09:45   
93483  31-03-2022      Vistara           UK            826          12:30   
93484  31-03-2022      Vistara           UK            832          07:05   
93485  31-03-2022      Vistara           UK            828          07:00   
93486  31-03-2022      Vistara           UK            822          09:45   

      departure_city flight_duration  \
0              Delhi  

## Standardize 'stops' column
economy and business dataframes have unclear values in the 'stops' column, parse them to integers (0, 1 or 2) 

In [51]:
# clean_df
stop_mapping = {
        'zero': 0,
        'one': 1,
        'two_or_more': 2
    }
    
clean_df['stops'] = clean_df['stops'].map(stop_mapping)

In [52]:
def stops_to_integers(df, column_name='stops'):
    """
    Maps the 'stops' column values to integers.
    - 0 for 'non-stop'.
    - 1 or 2+ based on the first number in the string for other values.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The column containing stop information.

    Returns:
        pd.DataFrame: DataFrame with the mapped stops column.
    """
    def map_stops(value):
        if 'non-stop' in value:
            return 0
        else:
            match = re.search(r'\d+', value) # extract first digit w regex
            return int(match.group()) if match else None

    df[column_name] = df[column_name].apply(map_stops)
    return df

In [53]:
stops_to_integers(business_df)

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,0,20:00,Mumbai,25612
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,0,21:15,Mumbai,25612
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1,20:45,Mumbai,42220
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1,23:55,Mumbai,44450
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1,23:55,Mumbai,46690
...,...,...,...,...,...,...,...,...,...,...,...
93482,31-03-2022,Vistara,UK,822,09:45,Chennai,10h 05m,1,19:50,Hyderabad,69265
93483,31-03-2022,Vistara,UK,826,12:30,Chennai,10h 25m,1,22:55,Hyderabad,77105
93484,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,79099
93485,31-03-2022,Vistara,UK,828,07:00,Chennai,10h 00m,1,17:00,Hyderabad,81585


In [54]:
stops_to_integers(economy_df)

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,0,21:05,Mumbai,5953
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,0,08:40,Mumbai,5953
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,0,06:35,Mumbai,5956
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,0,12:35,Mumbai,5955
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,0,11:10,Mumbai,5955
...,...,...,...,...,...,...,...,...,...,...,...
206769,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,7697
206770,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,7709
206771,31-03-2022,Vistara,UK,826,12:30,Chennai,20h 35m,1,09:05,Hyderabad,8640
206772,31-03-2022,Vistara,UK,822,09:45,Chennai,23h 20m,1,09:05,Hyderabad,8640


## Date & Time Conversion
- flight_date columns
- departure_time
- arrival_time

TBD: decide how to handle departure and arrival times in cleaned_df

In [55]:
functions.convert_date_column(economy_df, 'flight_date')

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,2022-02-11,SpiceJet,SG,8709,18:55,Delhi,02h 10m,0,21:05,Mumbai,5953
1,2022-02-11,SpiceJet,SG,8157,06:20,Delhi,02h 20m,0,08:40,Mumbai,5953
2,2022-02-11,AirAsia,I5,764,04:25,Delhi,02h 10m,0,06:35,Mumbai,5956
3,2022-02-11,Vistara,UK,995,10:20,Delhi,02h 15m,0,12:35,Mumbai,5955
4,2022-02-11,Vistara,UK,963,08:50,Delhi,02h 20m,0,11:10,Mumbai,5955
...,...,...,...,...,...,...,...,...,...,...,...
206769,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,7697
206770,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,7709
206771,2022-03-31,Vistara,UK,826,12:30,Chennai,20h 35m,1,09:05,Hyderabad,8640
206772,2022-03-31,Vistara,UK,822,09:45,Chennai,23h 20m,1,09:05,Hyderabad,8640


In [56]:
functions.convert_date_column(business_df, 'flight_date')

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,2022-02-11,Air India,AI,868,18:00,Delhi,02h 00m,0,20:00,Mumbai,25612
1,2022-02-11,Air India,AI,624,19:00,Delhi,02h 15m,0,21:15,Mumbai,25612
2,2022-02-11,Air India,AI,531,20:00,Delhi,24h 45m,1,20:45,Mumbai,42220
3,2022-02-11,Air India,AI,839,21:25,Delhi,26h 30m,1,23:55,Mumbai,44450
4,2022-02-11,Air India,AI,544,17:15,Delhi,06h 40m,1,23:55,Mumbai,46690
...,...,...,...,...,...,...,...,...,...,...,...
93482,2022-03-31,Vistara,UK,822,09:45,Chennai,10h 05m,1,19:50,Hyderabad,69265
93483,2022-03-31,Vistara,UK,826,12:30,Chennai,10h 25m,1,22:55,Hyderabad,77105
93484,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,79099
93485,2022-03-31,Vistara,UK,828,07:00,Chennai,10h 00m,1,17:00,Hyderabad,81585


In [57]:
functions.convert_time_column(business_df, 'arrival_time')
functions.convert_time_column(business_df, 'departure_time')

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,2022-02-11,Air India,AI,868,18:00,Delhi,02h 00m,0,20:00,Mumbai,25612
1,2022-02-11,Air India,AI,624,19:00,Delhi,02h 15m,0,21:15,Mumbai,25612
2,2022-02-11,Air India,AI,531,20:00,Delhi,24h 45m,1,20:45,Mumbai,42220
3,2022-02-11,Air India,AI,839,21:25,Delhi,26h 30m,1,23:55,Mumbai,44450
4,2022-02-11,Air India,AI,544,17:15,Delhi,06h 40m,1,23:55,Mumbai,46690
...,...,...,...,...,...,...,...,...,...,...,...
93482,2022-03-31,Vistara,UK,822,09:45,Chennai,10h 05m,1,19:50,Hyderabad,69265
93483,2022-03-31,Vistara,UK,826,12:30,Chennai,10h 25m,1,22:55,Hyderabad,77105
93484,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,79099
93485,2022-03-31,Vistara,UK,828,07:00,Chennai,10h 00m,1,17:00,Hyderabad,81585


In [58]:
functions.convert_time_column(economy_df, 'arrival_time')
functions.convert_time_column(economy_df, 'departure_time')

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price
0,2022-02-11,SpiceJet,SG,8709,18:55,Delhi,02h 10m,0,21:05,Mumbai,5953
1,2022-02-11,SpiceJet,SG,8157,06:20,Delhi,02h 20m,0,08:40,Mumbai,5953
2,2022-02-11,AirAsia,I5,764,04:25,Delhi,02h 10m,0,06:35,Mumbai,5956
3,2022-02-11,Vistara,UK,995,10:20,Delhi,02h 15m,0,12:35,Mumbai,5955
4,2022-02-11,Vistara,UK,963,08:50,Delhi,02h 20m,0,11:10,Mumbai,5955
...,...,...,...,...,...,...,...,...,...,...,...
206769,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,7697
206770,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,7709
206771,2022-03-31,Vistara,UK,826,12:30,Chennai,20h 35m,1,09:05,Hyderabad,8640
206772,2022-03-31,Vistara,UK,822,09:45,Chennai,23h 20m,1,09:05,Hyderabad,8640


## Generate Flight Code
combine 'airline_code' and 'flight_number' columns to generate 'flight_code' - business and economy dataframes, to match clean df

In [59]:
def create_flight_code(df):
    df['flight_code'] = df['airline_code'] + '-' + df['flight_number'].astype(str)
    return df

In [60]:
create_flight_code(business_df)

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price,flight_code
0,2022-02-11,Air India,AI,868,18:00,Delhi,02h 00m,0,20:00,Mumbai,25612,AI-868
1,2022-02-11,Air India,AI,624,19:00,Delhi,02h 15m,0,21:15,Mumbai,25612,AI-624
2,2022-02-11,Air India,AI,531,20:00,Delhi,24h 45m,1,20:45,Mumbai,42220,AI-531
3,2022-02-11,Air India,AI,839,21:25,Delhi,26h 30m,1,23:55,Mumbai,44450,AI-839
4,2022-02-11,Air India,AI,544,17:15,Delhi,06h 40m,1,23:55,Mumbai,46690,AI-544
...,...,...,...,...,...,...,...,...,...,...,...,...
93482,2022-03-31,Vistara,UK,822,09:45,Chennai,10h 05m,1,19:50,Hyderabad,69265,UK-822
93483,2022-03-31,Vistara,UK,826,12:30,Chennai,10h 25m,1,22:55,Hyderabad,77105,UK-826
93484,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,79099,UK-832
93485,2022-03-31,Vistara,UK,828,07:00,Chennai,10h 00m,1,17:00,Hyderabad,81585,UK-828


In [61]:
create_flight_code(economy_df)

,flight_date,airline_name,airline_code,flight_number,departure_time,departure_city,flight_duration,stops,arrival_time,arrival_city,price,flight_code
0,2022-02-11,SpiceJet,SG,8709,18:55,Delhi,02h 10m,0,21:05,Mumbai,5953,SG-8709
1,2022-02-11,SpiceJet,SG,8157,06:20,Delhi,02h 20m,0,08:40,Mumbai,5953,SG-8157
2,2022-02-11,AirAsia,I5,764,04:25,Delhi,02h 10m,0,06:35,Mumbai,5956,I5-764
3,2022-02-11,Vistara,UK,995,10:20,Delhi,02h 15m,0,12:35,Mumbai,5955,UK-995
4,2022-02-11,Vistara,UK,963,08:50,Delhi,02h 20m,0,11:10,Mumbai,5955,UK-963
...,...,...,...,...,...,...,...,...,...,...,...,...
206769,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,7697,UK-832
206770,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1,20:55,Hyderabad,7709,UK-832
206771,2022-03-31,Vistara,UK,826,12:30,Chennai,20h 35m,1,09:05,Hyderabad,8640,UK-826
206772,2022-03-31,Vistara,UK,822,09:45,Chennai,23h 20m,1,09:05,Hyderabad,8640,UK-822


#### Drop columns

In [62]:
business_df = business_df.drop(columns=['airline_code', 'flight_number'])
economy_df = economy_df.drop(columns=['airline_code', 'flight_number'])

## Generate Flight Time Groups
- business_df and economy_df have datetime information
- clean_df has time categories (Early morning, morning, etc.)
- Create group columns and assign time group according to predefined range

In [63]:
# function to generate time groups
def categorize_flight_time(time_str):
    hour = int(time_str.split(':')[0]) 
    
    # categorize based on the hour block
    if 4 <= hour < 6:
        return 'Early Morning'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 21:
        return 'Evening'
    elif 21 <= hour < 24:
        return 'Night'
    else:
        return 'Late Night'
    

# implementation

business_df['departure_time_group'] = business_df['departure_time'].apply(categorize_flight_time)
business_df['arrival_time_group'] = business_df['arrival_time'].apply(categorize_flight_time)

economy_df['departure_time_group'] = economy_df['departure_time'].apply(categorize_flight_time)
economy_df['arrival_time_group'] = economy_df['arrival_time'].apply(categorize_flight_time)

clean_df.rename(columns={'departure_time': 'departure_time_group', 'arrival_time' : 'arrival_time_group'}, inplace=True)

columns_to_clean = ['departure_time_group', 'arrival_time_group']

for col in columns_to_clean:
    clean_df[col] = clean_df[col].str.replace(' ', '', regex=True).str.replace('_', ' ', regex=True)

## Standardize Airline Names
For congruence in every dataframe

In [64]:
# mapping function for standardization
def standardize_airline_names(df, column_name='airline_name'):
    standard_name_mapping = {
        'SpiceJet': 'SpiceJet',
        'AirAsia': 'Air Asia',
        'Vistara': 'Vistara',
        'GO_FIRST': 'Go First',
        'GO FIRST': 'Go First',
        'Indigo': 'Indigo',
        'Air_India': 'Air India',
        'Air India': 'Air India',
        'Trujet': 'TruJet',
        'StarAir': 'Star Air',
    }
    
    df[column_name] = df[column_name].replace(standard_name_mapping)
    return df

# implementation
clean_df = standardize_airline_names(clean_df)
print(clean_df['airline_name'].unique())

business_df = standardize_airline_names(business_df)
print(clean_df['airline_name'].unique())

economy_df = standardize_airline_names(economy_df)
print(economy_df['airline_name'].unique())

['SpiceJet' 'Air Asia' 'Vistara' 'Go First' 'Indigo' 'Air India']
['SpiceJet' 'Air Asia' 'Vistara' 'Go First' 'Indigo' 'Air India']
['SpiceJet' 'Air Asia' 'Vistara' 'Go First' 'Indigo' 'Air India' 'TruJet'
 'Star Air']


## Format 'flight_duration' column
in clean_df to match the format of the other two

In [65]:
def format_duration(duration):
    hours = int(duration)
    minutes = round((duration - hours) * 60)
    return f"{hours:02}h {minutes:02}m"

clean_df['flight_duration'] = clean_df['flight_duration'].apply(format_duration)

#### Reorder columns

In [66]:
def reorder_columns(df, column_order):
    df = df[column_order]
    return df

column_order = [
    'flight_date', 'airline_name', 'flight_code', 
    'departure_time', 'departure_city', 
    'arrival_time', 'arrival_city', 
    'flight_duration', 'stops', 'price', 
    'departure_time_group', 'arrival_time_group'
]

In [67]:
business_df = reorder_columns(business_df, column_order)

In [68]:
economy_df = reorder_columns(economy_df, column_order)

In [69]:
clean_df = clean_df[[
    'airline_name', 'flight_code', 
    'departure_city', 
    'arrival_city', 
    'flight_duration', 'stops', 'price', 'class',
    'departure_time_group', 'arrival_time_group'
]]

### Check again

In [70]:
clean_df.head()

,airline_name,flight_code,departure_city,arrival_city,flight_duration,stops,price,class,departure_time_group,arrival_time_group
0,SpiceJet,SG-8709,Delhi,Mumbai,02h 10m,0,5953,Economy,Evening,Night
1,SpiceJet,SG-8157,Delhi,Mumbai,02h 20m,0,5953,Economy,Early Morning,Morning
2,Air Asia,I5-764,Delhi,Mumbai,02h 10m,0,5956,Economy,Early Morning,Early Morning
3,Vistara,UK-995,Delhi,Mumbai,02h 15m,0,5955,Economy,Morning,Afternoon
4,Vistara,UK-963,Delhi,Mumbai,02h 20m,0,5955,Economy,Morning,Morning


In [71]:
business_df.head()

,flight_date,airline_name,flight_code,departure_time,departure_city,arrival_time,arrival_city,flight_duration,stops,price,departure_time_group,arrival_time_group
0,2022-02-11,Air India,AI-868,18:00,Delhi,20:00,Mumbai,02h 00m,0,25612,Evening,Evening
1,2022-02-11,Air India,AI-624,19:00,Delhi,21:15,Mumbai,02h 15m,0,25612,Evening,Night
2,2022-02-11,Air India,AI-531,20:00,Delhi,20:45,Mumbai,24h 45m,1,42220,Evening,Evening
3,2022-02-11,Air India,AI-839,21:25,Delhi,23:55,Mumbai,26h 30m,1,44450,Night,Night
4,2022-02-11,Air India,AI-544,17:15,Delhi,23:55,Mumbai,06h 40m,1,46690,Afternoon,Night


In [72]:
economy_df.head()

,flight_date,airline_name,flight_code,departure_time,departure_city,arrival_time,arrival_city,flight_duration,stops,price,departure_time_group,arrival_time_group
0,2022-02-11,SpiceJet,SG-8709,18:55,Delhi,21:05,Mumbai,02h 10m,0,5953,Evening,Night
1,2022-02-11,SpiceJet,SG-8157,06:20,Delhi,08:40,Mumbai,02h 20m,0,5953,Morning,Morning
2,2022-02-11,Air Asia,I5-764,04:25,Delhi,06:35,Mumbai,02h 10m,0,5956,Early Morning,Morning
3,2022-02-11,Vistara,UK-995,10:20,Delhi,12:35,Mumbai,02h 15m,0,5955,Morning,Afternoon
4,2022-02-11,Vistara,UK-963,08:50,Delhi,11:10,Mumbai,02h 20m,0,5955,Morning,Morning


### Export Data Frames to .csv

In [ ]:
# clean_df.to_csv('../../data/clean/clean_dataset_df.csv', index=False)
# business_df.to_csv('../../data/clean/clean_business_df.csv', index=False)
# economy_df.to_csv('../../data/clean/clean_economy_df.csv', index=False)